In [1]:
import os
import sys
import logging
import datetime
import json
import numpy as np
import requests
import pandas as pd
from dotenv import load_dotenv

## Preparación

En esta celda se cargan las columnas a usar de la tabla CSV exportada de la hoja de cálculo

Los nombres de las columnas se renombran para que coincidan con los nombres de los endpoints del API de Directus

Se establece la base de la url del API. Si cambia la URL del API, cambiar la variable ```base_url```

In [2]:
load_dotenv()
config = json.load(open('../config/config.json'))

#archivo = config['xlsx']
archivo = config['csv']
key = os.getenv('KEY')
filesdir = config['files_dir']
headers =  {'Authorization':f"Bearer {key}"}
rename_dict = config['fields_map']
field_names = list(rename_dict.values())

print(field_names)
baseurl = config['baseurl']

obra_endpoint = 'items/obra/'
#campos many to one
m2o = config['m2o_map']

#campos manty to many
m2m = config['m2m_map']

upload_files = False


['arca_id', 'fuente_imagen', 'titulo', 'sintesis', 'anotacion_comentario_bibliografico', 'transcripcion_iconotexto', 'fechas_actividad']


In [3]:

if config['filtro']:
    excluir = pd.read_csv('../datos/salida/obras.csv', dtype=str)
    excluir
else:
    excluir = False



## Logs

Dataframes para reportes:

In [4]:
#df = pd.read_excel(archivo, dtype=object, sheet_name='Registro general' )
#df = pd.read_csv('arca_actualizada_RG.csv', dtype=str, usecols=colnames)
df = pd.read_csv(archivo, dtype=str)

In [5]:
print(f'-- rows loaded: {len(df.index)}')
df = df.rename(columns=rename_dict)

#eliminar filas vacias
df = df.dropna(how='all')
#eliminar filas sin id

df = df.dropna(subset=['arca_id'])
#rellenar NaNs
df = df.fillna('')

print(f'filas después de limpiar:{len(df.index)}')


# seleccion
#df = df.iloc[:22000]
#df.loc[df['arca_id'].astype(float).astype(int) <= 22340]
#print(df.iloc[-1])
#sample_data = df.sample(1)
#df = sample_data
#print(sample_data['t'].max())

obra = df

-- rows loaded: 24999
filas después de limpiar:24998


In [6]:
#print(len(excluir.index))# Filtrar los items ya subidos
if excluir:
    filtrados = df.loc[~df['arca_id'].isin(excluir['arca_id'].tolist())]
    print(f'-- Items despues de filtar:{len(filtrados.index)}')
    #filtrados = df[18736:]
    obra = filtrados

## Iteración

In [7]:
for index, row in obra.iterrows():
    print('----- NUEVA FILA -----')

    data = row.to_dict()
    #get obra data from table
    #este objeto va a ser posteado al api
    obra = row[field_names]
    obra = obra.to_dict()
    obra['arca_id'] = int(obra['arca_id'])
    arca_id = int(float(data['arca_id']))
    print(f'arca_id: {arca_id}', end='\n')
    print(obra)

    
    #get m2o objects and append
    for m2ofield in m2o:
        #columna con el id del campo relacionado
        idcol = m2ofield['arca_id']
        #obtener el id del campo, convertir a float e int
        if data[idcol]:
            resid = int(float(data[idcol]))
            #nombre del recurso a relacionar
            resourcename = m2ofield['resource']
            resourceurl = f'{baseurl}items/{resourcename}/?filter[arca_id][_eq]={resid}&limit=1'
            #print(resourceurl)
            try:
                response = requests.get(resourceurl)
                response.raise_for_status()

            except requests.RequestException as e:
                print(f'-- Error m2o GET related: {resourcename}--')
                print(e)
                continue

            print(f'-- OK GET {resourcename}--')

            #se saca el objeto de la respuesta
            try:
                object = response.json()['data'][0]
            #si no existe, continua a la siguiente iteracion
            except IndexError:
                print(f'-- Resultado vacío {resourceurl}--')
                continue
            
            obra[m2ofield['field']] = object
            #print(obra)

    # post obra
    print('-- POST obra -- ')
    #print('-- Obra  a crear --')
    #print(obra)

    # peticion para crear la obra
    try:
        r = requests.post(f'{baseurl}items/obra/', 
                       json=obra,
                      headers = headers)
        r.raise_for_status()

    except requests.RequestException as e:
        print(f'-- Error POST Obra: {arca_id}') 
        print(e)
        # item_ko = {
        #     'arca_id': arca_id,
        #     'resource': 'resourcename',
        #     'method': 'GET',
        #     'status': response.status_code,
        #     'error': e
        # }
        # items_ko.append(item_ko, ignore_index = True)
        continue

    print(f'-- OK POST Obra {r.status_code}--')
    #print(r.json())
    new_obra = r.json()['data']
    #print(new_obra)

    # item_ok = {
    #     'arca_id': arca_id,
    #     'api_id': new_obra['id'],
    #     'resource':'obra',
    #     'status': '200'
    # }


    for m2mfield in m2m:
        resourcename = m2mfield['resource']
        for obj in m2mfield['objects']:
            idcol = obj['arca_id']
            resid = data[idcol]
            #print(resid)
            #si existe id
            if resid:
                resid = int(float(resid))
                resourceurl = f'{baseurl}items/{resourcename}/?filter[arca_id][_eq]={resid}&limit=1'
                #print(resourceurl)

                try:
                    rel_res = requests.get(resourceurl)
                    rel_res.raise_for_status()

                except requests.RequestException as e:
                    print(f'-- Error m2m GET  related {resourcename}')
                    print(e)

                    # items_ok.append(item_ok)
                    # item_partial = item_ok
                    # item_partial['status'] = rel_res.status_code
                    # item_partial['resource'] = resourcename
                    # item_partial['method'] = 'GET'
                    # items_partial.append(item_partial, ignore_index=True)

                    continue

                print(f'-- OK GET {resourcename}--')

                try:
                    related = rel_res.json()['data'][0]
                except IndexError:
                    print(f'-- Resultado vacío {resourceurl}--')
                    continue

                #print('related')
                #print(related)
                #item_ok['resource'] = resourcename
                
                # Create relationship
                relation = {}
                relation['obra_id'] = new_obra['id']
                extfieldname = f'{resourcename}_id'
                #print(related['id'])
                relation[extfieldname] = related['id']
                #print(relation)
                # post relationship
                m2mresname = m2mfield['m2mresource']
                m2mresurl = f'{baseurl}items/{m2mresname}'

                try:
                    m2mresp = requests.post(m2mresurl, json=relation, headers=headers)
                    m2mresp.raise_for_status()
                    
                except requests.RequestException as e:
                    print(f'-- Error m2m POST relation {m2mresname}')
                    print(e)
                    # item_partial = item_ok
                    # item_partial['status'] = m2mresp.status_code
                    # item_partial['resource'] = resourcename
                    # item_partial['method'] = 'GET'
                    # items_partial.append(item_partial, ignore_index=True)
                    continue

                print(f'-- OK POST relation {m2mresname} --')
                print(f'OK CREACION OBRA: {arca_id}', end='\n')
                #print(m2mresp.json()['data'])
                #item_ok['resource'] = resourcename

            #column is empty, continue
            else:
                continue

    
    if upload_files:     
        # Crear archivo de url
        # Formatear id para 4 digitos
        idfmt = '{:04d}'.format(arca_id)
        #nombre de archivo
        filename = f'{idfmt}.jpg'
        filepath = filesdir + filename

        with open(filepath, 'rb') as infile:    
            print('open file')
            file = {
                'file': (filename,infile,'image/jpeg'),
            }
            
            files_endpoint = f'{baseurl}files'
            print('posting file')
            try:
                res_file = requests.post(files_endpoint, headers = headers, files=file)
                res_file.raise_for_status()

            except requests.RequestException as e:
                print(f'-- Error file POST {arca_id}--')
                print(e)
                # file_ko = {
                #     'arca_id': arca_id,
                #     'error': e
                # }
                # files_ko.append(file_ko)
                continue

            print('-- OK POST File --')
            nfile = res_file.json()['data']
            #print(nfile)
            # file_ok = {
            #     'arca_id': arca_id,
            #     'id': nfile['id'],
            #     'filename': filename
            # }
            # files_ok.append(file_ok, ignore_index=True)

            nfileid = nfile['id']
            obraid = new_obra['id']
            obraurl = f'{baseurl}items/obra/{obraid}'
            data = {
                'imagen':nfileid
            }
            try:
                updateres = requests.patch(obraurl, headers=headers, json=data)
                updateres.raise_for_status()

            except requests.RequestException as e:
                print(f'-- Error UPDATE imagen-obra {arca_id}')
                print(e)
                # item_partial['resource'] = updateres
                # item_partial['status'] = updateres.status_code
                # item_partial['error'] = e
                # item_partial['method'] = 'UPDATE'
                # items_partial.append(item_partial, ignore_index=True)
                continue

            print(f'-- OK UPDATE imagen-obra {arca_id}')
            #print(updateres.json()['data'])

            # item_ok['resource'] = updateres
            # item_ok['method'] = 'update'
            # item_ok['file'] = nfileid
            # items_ok.append(item_ok, ignore_index=True)


----- NUEVA FILA -----
arca_id: 2
{'arca_id': 2, 'fuente_imagen': 'Luis Mebold K. S.D.B. Catálogo de pintura colonial en Chile. Obras en monasterios de religiosas de Antigua fundación. Santiago: ediciones Universidad católica de Chile, 1987.', 'titulo': 'Desposorios de la Virgen y san José', 'sintesis': 'Formato vertical. Blanco y negro. Desposorio de la Virgen y San José. En el centro Sacerdote Abiathar (?) -sombrero de medialuna- pone su mano izquierda sobre las manos derechas de María (a la derecha del sacerdote) y José (a la izquierda del sacerdote).', 'anotacion_comentario_bibliografico': '', 'transcripcion_iconotexto': '', 'fechas_actividad': '1600-1699'}
-- OK GET autores--
-- Resultado vacío http://localhost:8056/items/autores/?filter[arca_id][_eq]=53&limit=1--
-- OK GET donante_lista--
-- Resultado vacío http://localhost:8056/items/donante_lista/?filter[arca_id][_eq]=6&limit=1--
-- OK GET tecnica_lista--
-- Resultado vacío http://localhost:8056/items/tecnica_lista/?filter[arca

KeyError: 'ubicacion_id'